In [2]:
import pandas as pd
import numpy as np

In [20]:
data_dir = '/Users/annagreene/code/district-comparison/data/'
dist1 = pd.read_csv(data_dir + 'raw/arizonaExample.csv')
mapping = pd.read_csv(data_dir + 'raw/final_mapping.csv')
dist2 = pd.read_csv(data_dir + 'raw/atlantaExample.csv')
replace = [',', '$', '-', '(', ')']
pd.options.mode.chained_assignment = None

In [16]:
#everything from clean tolleson script
dist1["Object_code"] = 0
dist1["Object_code"] = dist1.Account.str.split(".").str[3]

dist1 = dist1[dist1['Account Type']=="EXPENDITURE"]
dist1 = dist1.drop(columns=['Active', 'Account', 'Description', 'Account Type', 'Budget Control Group', 'Budget', 'Balance', 'Budget Balance', 'Encumbrance', 'Pre Encumbrance', 'Pending Invoices', 'Uncommitted Balance'])

dist1['YTD Transactions'] = dist1['YTD Transactions'].str.translate({ord(x): '' for x in replace}).astype(float)
dist1 = dist1.groupby(['Object_code'])['YTD Transactions'].sum().reset_index()
dist1 = dist1.rename(columns={'YTD Transactions': 'Transactions'})
dist1 = dist1.round(2)

dist1 = dist1.rename(columns={'Object_code': 'Code'})
dist1['Code'] = dist1['Code'].astype(str)
dist1['Code'] = dist1['Code'].str.strip()
display(dist1)

,Code,Transactions
0,6000,0.00
1,6100,0.00
2,6101,0.00
3,6103,0.00
4,6110,8002.11
...,...,...
181,6890,455508.18
182,6891,0.00
183,6892,0.00
184,6893,0.00


In [21]:
#everything from clean atlanta script
dist2 = dist2[~dist2.DESCRIPTION.str.contains('TOTAL')]
dist2 = dist2.drop(columns=['Unnamed: 1', 'DESCRIPTION'])
dist2 = dist2.rename(columns={'ACCOUNT': 'Code', ' AMOUNT ': 'Transactions'})

dist2['Code'] = dist2['Code'].str[5:8]
dist2['Code'] = dist2['Code'].astype(str)

dist2['Transactions'] = dist2['Transactions'].str.translate({ord(x): '' for x in replace})
dist2['Transactions'] = np.double(dist2['Transactions'])
dist2 = dist2.groupby(['Code'])['Transactions'].sum().reset_index()
dist2 = dist2.round(2)

display(dist2)

,Code,Transactions
0,110,2.254804e+08
1,111,1.446570e+05
2,112,5.384203e+06
3,113,5.684628e+06
4,114,5.072093e+05
...,...,...
95,880,2.814457e+06
96,881,4.291009e+07
97,882,3.109448e+06
98,890,3.051903e+06


In [22]:
#everything from clean mapping script
cols = ["Object_code","Function_code", "Category_Edstruments","Major_Category_Edstruments","Title", "State"]
clean_map = mapping[cols]
clean_map = clean_map.fillna(0)

clean_map["Code"] = ''
clean_map["Code"] = np.where(clean_map["Object_code"] == 0, clean_map["Function_code"], clean_map["Object_code"])
clean_map["Code"] = clean_map["Code"].astype(str)
clean_map["Code"] = clean_map["Code"].str.strip()
#clean_map = clean_map[clean_map["Code"].str.contains('X')==False]
#clean_map["Code"] = clean_map["Code"].astype(str)

display(clean_map)

,Object_code,Function_code,Category_Edstruments,Major_Category_Edstruments,Title,State,Code
0,10,0,Teachers Salaries,Salaries,Regular Teacher,Alabama,10
1,11,0,Teachers Salaries,Salaries,Resource Teacher,Alabama,11
2,12,0,Teachers Salaries,Salaries,Alternative School Teacher,Alabama,12
3,14,0,Teachers Salaries,Salaries,Teacher - Gifted,Alabama,14
4,15,0,Special Education Salaries,Salaries,Teacher - Collaborative Special Education,Alabama,15
...,...,...,...,...,...,...,...
7962,728,0,Miscellaneous,Other Financing Uses,Transfers to Agency Funds,Wyoming,728
7963,729,0,Miscellaneous,Other Financing Uses,Transfers to Any Other Funds,Wyoming,729
7964,730,0,Miscellaneous,Other Financing Uses,Payments to Escrow Agents,Wyoming,730
7965,740,0,Debt Service,Other Financing Uses,Redemption of Principal Other,Wyoming,740


In [85]:
#most of the stuff from map script
mapping = pd.read_csv(data_dir + 'processed/mapping.csv')
dist_1_og = pd.read_csv(data_dir + 'processed/tolleson.csv')
state_1 = 'Arizona'
dist_2_og = pd.read_csv(data_dir + 'processed/atlanta.csv')
state_2 = 'Georgia'

dist_1_map = mapping[mapping['State'] == state_1].reset_index()
dist_1_map = dist_1_map.drop(columns=['index', 'Unnamed: 0', 'Object_code', 'Function_code'])
dist_1_map.loc[len(dist_1_map.index)] = ['Miscellaneous', 'Other Financing Uses', '*Unable to assign to mapping tool*', state_1, 'XXXX']

dist_2_map = mapping[mapping['State'] == state_2].reset_index()
dist_2_map = dist_2_map.drop(columns=['index', 'Unnamed: 0', 'Object_code', 'Function_code'])
dist_2_map.loc[len(dist_2_map.index)] = ['Miscellaneous', 'Other Financing Uses', '*Unable to assign to mapping tool*', state_2, 'XXXX']

dist_1_og['Code'] = dist_1_og['Code'].astype(str)
dist_2_og['Code'] = dist_2_og['Code'].astype(str)

dist_1_merge_diff = dist_1_og.merge(dist_1_map, how='left', on='Code', indicator=True)
dist_1_merge_diff2 = dist_1_merge_diff[dist_1_merge_diff['_merge'] == 'left_only']
dist_1_not_mapped = list(dist_1_merge_diff2['Code'])
dist_1_og['Code'] = np.where(dist_1_og['Code'].isin(dist_1_not_mapped), dist_1_og['Code'] == 'False', dist_1_og['Code'])

dist_1_og.loc[(dist_1_og.Code == False),'Code']='XXXX'
dist_1_og = dist_1_og.groupby(['Code'])['Transactions'].sum().reset_index()

dist_1_merge = dist_1_og.merge(dist_1_map, how='left', on='Code')
dist_1_merge

,Code,Transactions,Category_Edstruments,Major_Category_Edstruments,Title,State,_merge
0,6100,0.00,Other Salaries,Salaries,PERSONAL SERVICES - SALARIES,Arizona,both
1,6110,8002.11,Other Salaries,Salaries,Certified Salaries,Arizona,both
2,6111,4490251.59,Administrative Salaries,Salaries,Administrators,Arizona,both
3,6112,35105711.03,Teachers Salaries,Salaries,Teachers,Arizona,both
4,6113,398413.43,Teachers Salaries,Salaries,Substitute Teachers,Arizona,both
...,...,...,...,...,...,...,...
108,6842,313302.52,Miscellaneous,Other Financing Uses,Other,Arizona,both
109,6850,0.00,Debt Service,Other Financing Uses,Interest on Short-Term Debt,Arizona,both
110,6860,26612.14,Debt Service,Other Financing Uses,Bond Issuance and Other Debt-Related Costs,Arizona,both
111,6890,455508.18,Miscellaneous,Other Financing Uses,Miscellaneous Expenditures,Arizona,both
